In [ ]:
import pandas as pd
import numpy as np

import MySQLdb
from pandas.io import sql

conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor() 

In [ ]:
df_concept = pd.read_sql('select * from `concept`', con = conn) 
df_concept

# Getting numeric lab concept

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()
df_labevents_mimic_num = pd.read_sql("SELECT * from mimic4.d_labitems where itemid IN (SELECT distinct itemid from mimic4.labevents where value NOT regexp '[[:alpha:]]')", con = mimicconn)

In [ ]:
df_labevents_mimic_num['label'] = df_labevents_mimic_num['label'].str.lower()
df_labevents_mimic_num = df_labevents_mimic_num.drop_duplicates(subset='label', keep='first')

In [ ]:
df_labevents_mimic_num[df_labevents_mimic_num['label']=='potassium']

In [ ]:
itemd_list = df_labevents_mimic_num["itemid"].tolist()
itemd_list

In [ ]:
df_concept_labevents = df_labevents_mimic_num.rename({"loinc_code": "short_name", "label": "description" }, axis='columns')
df_concept_labevents_ = df_concept_labevents.drop(columns=['itemid', 'fluid', 'category'])
df_concept_labevents_['date_created']= '2022-01-01'
import uuid
df_concept_labevents_['uuid'] = [uuid.uuid4() for x in range(len(df_concept_labevents_))]
df_concept_labevents_['datatype_id'] = 1
df_concept_labevents_['class_id'] = 1
df_concept_labevents_['creator']=1


# Getting nonnumeric lab concepts

In [ ]:
df_labevents_mimic_nonnum = pd.read_sql("SELECT * from mimic4.d_labitems where itemid IN (SELECT distinct itemid from mimic4.labevents where value regexp '[[:alpha:]]')", con = mimicconn)
df_labevents_mimic_nonnum

In [ ]:
df_labevents_mimic_nonnum['label'] = df_labevents_mimic_nonnum['label'].str.lower()
df_labevents_mimic_nonnum = df_labevents_mimic_nonnum.drop_duplicates(subset='label', keep='first')


In [ ]:
df_labevents_mimic_nonnum

In [ ]:
df_labevents_mimic_nonnum[df_labevents_mimic_nonnum['label']=='cd20']

In [ ]:
df_labevents_mimic_nonnum_ = pd.read_sql("SELECT distinct itemid, valueuom, ref_range_lower, ref_range_upper from mimic4.labevents where itemid IN (SELECT distinct itemid from mimic4.labevents where value regexp '[[:alpha:]]')", con = mimicconn)


In [ ]:
df_labevents_mimic_nonnum_d = df_labevents_mimic_nonnum_.sort_values(by = 'valueuom', na_position='last', ascending=False)
df_labevents_mimic_nonnum_d = df_labevents_mimic_nonnum_d.drop_duplicates(subset='itemid', keep='first')
df_labevents_mimic_nonnum_d 

In [ ]:
df_concept_labevents_nonnum = df_labevents_mimic_nonnum.rename({"loinc_code": "short_name", "label": "description" }, axis='columns')
df_concept_labevents_nonnum = df_concept_labevents_nonnum.drop(columns=['itemid', 'fluid', 'category'])
df_concept_labevents_nonnum['date_created']= '2022-01-01'
import uuid
df_concept_labevents_nonnum['uuid'] = [uuid.uuid4() for x in range(len(df_concept_labevents_nonnum))]
df_concept_labevents_nonnum['datatype_id'] = 2
df_concept_labevents_nonnum['class_id']= 1
df_concept_labevents_nonnum['creator'] = 1

In [ ]:
total_lab_concept = df_concept_labevents_.append(df_concept_labevents_nonnum)
total_lab_concept

In [ ]:
total_lab_concept = total_lab_concept.drop_duplicates(subset = 'description', keep = 'first')
total_lab_concept


In [ ]:
column_names = ['short_name', 'description', 'datatype_id', 'class_id', 'creator', 'date_created', 'uuid']
total_lab_concept = total_lab_concept.reindex(columns=column_names)
total_lab_concept

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in total_lab_concept.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in total_lab_concept.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

In [ ]:
total_lab_concept[total_lab_concept['datatype_id']==1]

# Dealing with concept numeric

In [ ]:
df_concept_=pd.read_sql('select * from `concept` where datatype_id=1 and class_id = 1', con = conn) 
df_concept_

In [ ]:
conceptid_list = df_concept_["concept_id"].tolist()
len(conceptid_list)

In [ ]:
itemid_conceptid = dict(zip(itemd_list, conceptid_list))
itemid_conceptid

In [ ]:
df_labevents_mimic_num_ = pd.read_sql("SELECT distinct itemid, valueuom, ref_range_lower, ref_range_upper from mimic4.labevents where itemid IN (SELECT distinct itemid from mimic4.labevents where value NOT regexp '[[:alpha:]]')", con = mimicconn)


In [ ]:
df_labevents_mimic_num_ = df_labevents_mimic_num_.sort_values(by = 'valueuom', na_position='last', ascending=False)
df_labevents_mimic_num_ = df_labevents_mimic_num_.drop_duplicates(subset='itemid', keep='first')
df_labevents_mimic_num_ 

In [ ]:
df_labevents_mimic_num_ = pd.merge(df_labevents_mimic_num, df_labevents_mimic_num_, on="itemid")
df_labevents_mimic_num_

In [ ]:
df_labevents_mimic_num_['concept_id'] = df_labevents_mimic_num_['itemid'].map(itemid_conceptid) 
df_labevents_mimic_num_

In [ ]:
df_labevents_mimic_num_copy = df_labevents_mimic_num_[['valueuom', 'ref_range_lower', 'ref_range_upper', 'concept_id']]
df_labevents_mimic_num_copy= df_labevents_mimic_num_copy.rename(columns={"valueuom": "units", "ref_range_lower":"low_normal", "ref_range_upper": "hi_normal"})
df_labevents_mimic_num_copy

In [ ]:
df_labevents_mimic_num_copy= df_labevents_mimic_num_copy.replace({np.NAN: None})
df_labevents_mimic_num_copy

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in df_labevents_mimic_num_copy.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_labevents_mimic_num_copy.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept_numeric` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 


In [ ]:
df_labevents_nonnum_merge = pd.merge(df_labevents_mimic_nonnum, df_labevents_mimic_nonnum_d, on="itemid")
df_labevents_nonnum_merge

# Dealing with coded answer

In [ ]:
_df_ = pd.read_csv('non-numeric-labevents_.csv')

_df_= _df_[['value', 'value1']]
_df_.dropna(inplace = True)
_df_

In [ ]:
dict_nonnum_1 = dict(zip(_df_.value, _df_.value1))
dict_nonnum_1

In [ ]:
a = set(_df_['value1'].to_list())
len(a)

In [ ]:
b = [item for item in a if not item.isdigit()]

In [ ]:
b.remove('274.9')

In [ ]:
for index, row in _df_.iterrows():
    if row['value1'] not in b:
        _df_=_df_.drop(index)

In [ ]:
dict_nonnum = dict(zip(_df_.value, _df_.value1))
dict_nonnum

In [ ]:
_df_cod_concept = _df_[['value1']]
_df_cod_concept = _df_cod_concept.rename({'value1':'description'}, axis='columns')

In [ ]:
unique_coded_answer = _df_cod_concept.sort_values(by = 'description', na_position='last', ascending=False)
unique_coded_answer = unique_coded_answer.drop_duplicates(subset='description', keep='first')
unique_coded_answer 

In [ ]:
unique_coded_answer['description'].nunique()

In [ ]:
unique_coded_answer['date_created']= '2022-01-01'
import uuid
unique_coded_answer['uuid'] = [uuid.uuid4() for x in range(len(unique_coded_answer))]
unique_coded_answer

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in unique_coded_answer.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in unique_coded_answer.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 

datatype_id = 'UPDATE concept SET datatype_id=4 where datatype_id=0'
class_id = 'UPDATE concept SET class_id=11 where class_id=0'
concept_creator = 'UPDATE concept SET creator = 1' 

openmrscursor.execute(datatype_id)
openmrscursor.execute(class_id)
openmrscursor.execute(concept_creator)


conn.commit() 

# Dealing with importing into concept answer table

In [ ]:
unique_cod_values = pd.read_sql("SELECT distinct itemid, value FROM mimic4.labevents where value regexp '[[:alpha:]]'", con=mimicconn)
unique_cod_values

In [ ]:
unique_cod_values_ = unique_cod_values.drop_duplicates(keep='first')
unique_cod_values_ 

In [ ]:
df_labitems_mimic_ = pd.read_sql("SELECT itemid, label from mimic4.d_labitems where itemid IS NOT NULL AND label IS NOT NULL", con = mimicconn)
df_labitems_mimic_

In [ ]:
df_labitems_mimic_ = df_labitems_mimic_.drop_duplicates(keep= 'first')
df_labitems_mimic_

In [ ]:
dict_nonnum_1

In [ ]:
unique_cod_values_['value1'] = unique_cod_values_['value'].map(dict_nonnum_1)
unique_cod_values_

In [ ]:
unique_cod_values_[unique_cod_values_['value1'].isnull()]

In [ ]:
List = set(unique_cod_values_['value1'].to_list())
List

In [ ]:
num = ['1125',
 '1265',
 '1440',
 '1840',
 '1850',
 '233',
 '257',
 '274.9',
 '3553',
 '610',
 '642',
 '660',
 '680',
 '685',
 '770',
 '820',
 '834',
 '875',
 '915',
 '936']
unique_cod_values_ = unique_cod_values_[~unique_cod_values_['value1'].isin(num)]
unique_cod_values_

In [ ]:
unique_cod_values_[unique_cod_values_['value1'].isnull()]

In [ ]:
unique_cod_values_1 = pd.merge(df_labitems_mimic_, unique_cod_values_, on="itemid")
unique_cod_values_1

In [ ]:
unique_cod_values_1.isnull().sum()

In [ ]:
df_nonnum_con = pd.read_sql('select * from concept where class_id = 1', con=conn)
df_nonnum_con

In [ ]:
df_con_ans = pd.read_sql('select * from concept where class_id = 11', con=conn)
df_con_ans 

In [ ]:
description_concept = dict(zip(df_nonnum_con.description, df_nonnum_con.concept_id))
len(description_concept)

In [ ]:
unique_cod_values_1['label'] = unique_cod_values_1['label'].str.lower()

In [ ]:
unique_cod_values_1['concept_id'] = unique_cod_values_1['label'].map(description_concept)
unique_cod_values_1

In [ ]:
description_concept2 = dict(zip(df_con_ans.description, df_con_ans.concept_id))
description_concept2

In [ ]:
unique_cod_values_1['answer_concept'] = unique_cod_values_1['value1'].map(description_concept2)
unique_cod_values_1

In [ ]:
unique_cod_values_1 = unique_cod_values_1.drop(columns=['itemid', 'label', 'value'])
unique_cod_values_1['date_created']= '2022-01-01'
unique_cod_values_1['uuid'] = [uuid.uuid4() for x in range(len(unique_cod_values_))]
unique_cod_values_1

In [ ]:
unique_cod_values_1['creator'] = 1

In [ ]:
column_names = ['concept_id', 'answer_concept', 'creator', 'date_created', 'uuid']
unique_cod_values_1 = unique_cod_values_1.reindex(columns=column_names)

In [ ]:
# creating column list for insertion into concept table
cols = "`,`".join([str(i) for i in unique_cod_values_1.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in unique_cod_values_1.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `concept_answer` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        openmrscursor.execute(sql1)
        openmrscursor.execute(sql, tuple(row))
        openmrscursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit() 
